In [1]:
# importação de pacotes
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
print("Versão do TensorFlow: ", tf.__version__)

import PIL
import imageio
from IPython import display

Versão do TensorFlow:  2.7.0


In [2]:
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

DATA_FOLDERS = ["tiny-hero", "rpg-maker"]
DATASET_SIZE = int(sum([3648, 4064]))
BATCH_SIZE = 64 
IMAGE_SIZE = 64
CHANNELS = 4
EPOCHS = 50

In [3]:
class_names = []
X_train = [0, 1]
for i in range(2):
    X_train[i] = keras.preprocessing.image_dataset_from_directory(
        DATA_FOLDERS[i],
        class_names = ["0-back", "1-left", "2-front", "3-right"],
        label_mode= "int",
        validation_split=0.2,
        subset="training",
        color_mode="rgba",
        interpolation="nearest",
        seed=SEED,
        batch_size=BATCH_SIZE,
        image_size=(IMAGE_SIZE, IMAGE_SIZE))
    class_names = X_train[i].class_names
X_train = X_train[0].concatenate(X_train[1])
X_train_size = tf.math.ceil(DATASET_SIZE * 0.8).numpy().astype(int) + 1
print("X_train_size", X_train_size)

X_validation = [0, 1]
for i in range(2):
    X_validation[i] = keras.preprocessing.image_dataset_from_directory(
        DATA_FOLDERS[i],
        class_names = ["0-back", "1-left", "2-front", "3-right"],
        label_mode= "int",
        validation_split=0.2,
        subset="validation",
        color_mode="rgba",
        interpolation="nearest",
        seed=SEED,
        batch_size=BATCH_SIZE,
        image_size=(IMAGE_SIZE, IMAGE_SIZE))
X_validation = X_validation[0].concatenate(X_validation[1])
X_validation_size = DATASET_SIZE - X_train_size
print("X_validation_size", X_validation_size)

normalization_layer = keras.layers.experimental.preprocessing.Rescaling(1./127.5, -1)
X_train = X_train.map(lambda x, y: (normalization_layer(x), y)).shuffle(DATASET_SIZE)
X_validation = X_validation.map(lambda x, y: (normalization_layer(x), y))

FileNotFoundError: [WinError 3] O sistema não pode encontrar o caminho especificado: 'tiny-hero'

In [ ]:
def denormalize_image(image):
    return (image + 1) / 2

def show_image(image):
    plt.imshow(image, interpolation="nearest")
    
def show_normalized_image(image):
    show_image(denormalize_image(image))

plt.figure(figsize=(8, 9))
plt.suptitle("Exemplos do treinamento")
plt.subplots_adjust(bottom=0)
for i in range(4):
    X_train_by_class = X_train.filter(lambda x, l: tf.math.equal(l[0], i))
    for j in range(4):
        for images, labels in X_train_by_class.take(1):
            ax = plt.subplot(4, 4, (i*4+j)+1)
            show_normalized_image(images[0])
            plt.title(class_names[labels[0]])
            plt.axis("off")

In [ ]:
X_train = X_train.cache().shuffle(DATASET_SIZE).prefetch(buffer_size=tf.data.AUTOTUNE)
X_validation = X_validation.cache().prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
pose_classifier = keras.Sequential([
    layers.Conv2D(2, 3, padding="same", activation="relu", input_shape=(IMAGE_SIZE, IMAGE_SIZE, CHANNELS)),
    layers.Conv2D(8, 3, strides=2, padding="same", activation="relu"),
    layers.BatchNormalization(),
    layers.Dropout(0.8),
    
    layers.Flatten(),
    layers.Dense(128),
    layers.Dense(len(class_names), activation="softmax")
#     layers.Conv2D(32, 3, padding="same", activation="relu", input_shape=(IMAGE_SIZE, IMAGE_SIZE, CHANNELS)),
#     layers.MaxPool2D(),
#     layers.Dropout(0.8),

#     layers.Flatten(),
#     layers.Dense(24),
#     layers.Dense(len(class_names), activation="softmax")
])

pose_classifier.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

pose_classifier.summary()

In [ ]:
history = pose_classifier.fit(
  X_train,
  validation_data=X_validation,
  epochs=EPOCHS,
)

In [ ]:
acc = history.history["accuracy"]
val_acc = history.history["val_accuracy"]

loss = history.history["loss"]
val_loss = history.history["val_loss"]

epochs_range = range(len(loss))

plt.figure(figsize=(16, 6))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label="Acurácia (treinamento)")
plt.plot(epochs_range, val_acc, label="Acurácia (validação)")
plt.legend(loc="lower right")
plt.grid()
plt.title("Acurácia de treino e validação")

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label="Erro no treino")
plt.plot(epochs_range, val_loss, label="Erro na validação")
plt.legend(loc="upper right")
plt.title("Erro de treino e validação")
plt.grid()
plt.show()

In [ ]:
len(loss)

In [ ]:
Y_predicted = np.argmax(pose_classifier.predict(X_validation), axis=1).astype(int).reshape(-1, 1)
X_validation_unbatched = np.array(list(X_validation.unbatch().as_numpy_iterator()), dtype=object)
X_validation_unbatched = np.concatenate((X_validation_unbatched, Y_predicted), axis=1)

X_validation_by_class = {name:[] for name in class_names}
print(X_validation_by_class)

total = 0
equal = 0
for (image, y, y_hat) in X_validation_unbatched:
    total += 1
    equal += 1 if y == y_hat else 0
    X_validation_by_class[class_names[y]].append((image, y, y_hat))
    
print('total', total)
print('equal', equal)
    
plt.figure(figsize=(10, 11))
plt.suptitle("Algumas previsões")
plt.subplots_adjust(bottom=0)
for i, class_name in enumerate(class_names):
    indices = np.arange(len(X_validation_by_class[class_names[i]]), dtype="int")
    indices = np.random.choice(indices, size=4, replace=False)
    examples_from_current_class = np.array(X_validation_by_class[class_names[i]], dtype=object)[indices]
    for j in range(4):
        ax = plt.subplot(4, 4, (i*4+j)+1)
        show_normalized_image(examples_from_current_class[j][0])
        plt.title(class_names[examples_from_current_class[j][2]])
        plt.axis("off")

## Estudando previsões incorretas

In [ ]:
plt.figure(figsize=(10, 10))
plt.suptitle("Algumas previsões erradas")
plt.subplots_adjust(bottom=0)

error_examples = {class_name:[(image, y, y_hat) for (image, y, y_hat) in examples if y != y_hat] for (class_name, examples) in X_validation_by_class.items()}

for i, class_name in enumerate(class_names):
    error_examples_from_current_class = error_examples[class_name]
    number_of_cells = min(4, len(error_examples_from_current_class))
    for j in range(number_of_cells):
        ax = plt.subplot(4, 4, (i*4+j)+1)
        show_normalized_image(error_examples_from_current_class[j][0])
        plt.title(f"Real: {class_names[error_examples_from_current_class[j][1]]}\nPred: {class_names[error_examples_from_current_class[j][2]]}")
        plt.axis("off")
